In [1]:
from code_tokenizer import *

code = """
a = 1
for(i = 0; i < 10; i++) {
    a = a + 1
}
"""
print(color_by_char_type(code))
parsed = parse(code)
def read(parsed):
    if(type(parsed[0]) != list):
        parsed = [parsed]
    for item in parsed:
        if(item[0] == 'assign'):
            obj, val = item[1]
            print(obj, val)
        elif(item[0] == 'if'):
            print('if', item[1])
read(parsed)
print()
for item in parsed:
    print(item)


a = 1
for(i = 0; i < 10; i++) {
    a = a + 1
}

a ['number', '1']

['assign', ['a', ['number', '1']]]
['for', [[['assign', [['word', 'i'], ['number', '0']]], ['operation', '<', [['word', 'i'], ['number', '10']]], [['increment', ['word', 'i']]]], ['assign', ['a', ['operation', '+', [['word', 'a'], ['number', '1']]]]]]]
